In [74]:
#Disciplina: Solucoes em Mineracao de dados
#--------------------------------------------------------
#Script para a analise exploratoria dos dados (AED)
#--------------------------------------------------------

# Importando as bibliotecas necessarias

!pip3 install xlrd
!pip3 install scikit-learn

import pandas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Importação do dataset

Por meio da opção dtype, na leitura do arquivo xls, é possível apontar o tipo exato de cada atributo, garantindo um maior controle sobre o dataset.


Foram criadas listas de nomes de atributos, para auxiliar na separação dos datasets de treino e teste.

- features: Lista com todos os atributos que não são a label.
- label: Lista contendo o atributo que é a label.

In [75]:
file_path = "./default of credit card clients.xls"

dtypes = {
    "ID": int,
    "LIMIT_BAL": float,
    "SEX": "object",                       
    "EDUCATION": "object",                     
    "MARRIAGE": "object",                
    "AGE": int,                          
    "PAY_0": "object",                        
    "PAY_2": "object",                        
    "PAY_3": "object",                         
    "PAY_4": "object",                        
    "PAY_5": "object",                         
    "PAY_6": "object",                        
    "BILL_AMT1": float,                    
    "BILL_AMT2": float,              
    "BILL_AMT3": float,              
    "BILL_AMT4": float,              
    "BILL_AMT5": float,              
    "BILL_AMT6": float,              
    "PAY_AMT1": float,                      
    "PAY_AMT2": float,                      
    "PAY_AMT3": float,                      
    "PAY_AMT4": float,                      
    "PAY_AMT5": float,                      
    "PAY_AMT6": float,                      
    "default payment next month": "object"
}

df = pandas.read_excel(file_path, header = 1, dtype=dtypes)

columns = list(df.columns)
features = columns[:len(columns)-1]
label = columns[len(columns)-1:]

# Verificando tipos de dados

In [76]:
df.dtypes

ID                              int64
LIMIT_BAL                     float64
SEX                            object
EDUCATION                      object
MARRIAGE                       object
AGE                             int64
PAY_0                          object
PAY_2                          object
PAY_3                          object
PAY_4                          object
PAY_5                          object
PAY_6                          object
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default payment next month     object
dtype: object

# Separando dataset entre treinamento e teste
Os datasets serão divididos entre conjuntos de treino (75%) e teste (25%).

random_state é setado para 42 para garantir que sempre que esse bloco seja executado, resulte na mesma amostragem, afim de possibilitar reprodução exata dos resultados (já que a função train_test_split usa métodos "aleatórios" para gerar as amostras).

In [77]:
X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Removendo colunas desnecessárias
Foi considerado que a coluna "ID" não tem utilidade, e representa um atributo com valores únicos.

Uma lista com todas os atributos úteis é criada e o conjunto de treino é podado com base nela.

In [78]:
useful_features = [
    'LIMIT_BAL',
    'SEX',
    'EDUCATION',
    'MARRIAGE',
    'AGE',
    'PAY_0',
    'PAY_2',
    'PAY_3',
    'PAY_4',
    'PAY_5',
    'PAY_6',
    'BILL_AMT1',
    'BILL_AMT2',
    'BILL_AMT3',
    'BILL_AMT4',
    'BILL_AMT5',
    'BILL_AMT6',
    'PAY_AMT1',
    'PAY_AMT2',
    'PAY_AMT3',
    'PAY_AMT4',
    'PAY_AMT5',
    'PAY_AMT6'
]

X_train = X_train[useful_features]

# Preparar uma base de dados no formato numérico
A base de dados já encontra-se em formato numérico. Os dados categóricos, embora codificados como números, não representam valores, mas sim categorias/classes. Esse fato é tratado na codificação de dados categóricos.

# Preenchendo dados ausentes

Por meio da função do Pandas isnull() e sum(), é possível contabilizar quantos dados ausentes existem em cada atributo.
Nenhum dado ausente foi encontrado.

In [79]:
X_train.isnull().sum()

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
dtype: int64

# Codificando dados categóricos em dados numéricos
Usando a função get_dummies do Pandas, é possível aplicar one-hot-encoding nos atributos categóricos,
gerando uma maneira de determinar a categoria dos registros de uma forma que os modelos entendam.

In [80]:
categorical_features = [
    'SEX',
    'EDUCATION',
    'MARRIAGE',
    'PAY_0',
    'PAY_2',
    'PAY_3',
    'PAY_4',
    'PAY_5',
    'PAY_6',
]

X_train = pandas.get_dummies(X_train, prefix=categorical_features, columns=categorical_features)

# Escalar dados numéricos com min-max
Foi criada uma função "normalize_df_columns" para aplicar a normalização nos atributos numéricos.

In [81]:
def normalize_df_columns(df: pandas.DataFrame, columns: list):
    df[columns] = (
        (df[columns] - df[columns].min())
        /
        (df[columns].max()-df[columns].min())
    )

    return df

numerical_features = [
    'LIMIT_BAL',
    'AGE',
    'BILL_AMT1',
    'BILL_AMT2',
    'BILL_AMT3',
    'BILL_AMT4',
    'BILL_AMT5',
    'BILL_AMT6',
    'PAY_AMT1',
    'PAY_AMT2',
    'PAY_AMT3',
    'PAY_AMT4',
    'PAY_AMT5',
    'PAY_AMT6'
]

X_train = normalize_df_columns(X_train, numerical_features)

# Resultado final

In [82]:
X_train.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_6_-2,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8
21177,0.292929,0.185185,0.218131,0.144696,0.084590,0.241396,0.167150,0.254739,0.004579,0.002375,...,0,0,1,0,0,0,0,0,0,0
23942,0.010101,0.055556,0.160438,0.082141,0.055947,0.173979,0.097419,0.190589,0.001503,0.001253,...,0,0,1,0,0,0,0,0,0,0
1247,0.080808,0.259259,0.148879,0.068751,0.035643,0.160138,0.080648,0.178567,0.003053,0.000000,...,1,0,0,0,0,0,0,0,0,0
23622,0.292929,0.351852,0.146519,0.066220,0.035643,0.160138,0.080648,0.178567,0.000000,0.000000,...,1,0,0,0,0,0,0,0,0,0
28454,0.060606,0.277778,0.206716,0.130625,0.069930,0.187751,0.109249,0.203717,0.003823,0.001214,...,0,0,1,0,0,0,0,0,0,0
